### Code to Perform Computation Required to Create GridMap with TimeSlider
This must be run on Compute Canada. Data is big. It can be found in the remote directory:
sahakhsh@cedar.computecanada.ca:~/scratch/Grid_with_TimeSlider_Processing

#### Import libraries

In [63]:
import branca.colormap as cm
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import math

#### Combine tweet data by day to create one big file

In [64]:
def get_filename(day):
    if day < 10:
        return r"./processed_geotweets/corona_geotweets_0" + str(day) + r"_data.csv"
    return r"./processed_geotweets/corona_geotweets_" + str(day) + r"_data.csv"

total_days = 350
total_weeks = int(math.ceil(total_days/7))
curr_day_data = pd.read_csv("./processed_geotweets/corona_geotweets_01_data.csv")
combined_geotweet_df = curr_day_data.copy()
combined_geotweet_df['day'] = 1

for day in range(2,total_days+1):
    curr_day_df = pd.read_csv(get_filename(day), lineterminator='\n', low_memory=False)    
    curr_day_df['day'] = day
    combined_geotweet_df = pd.concat([combined_geotweet_df, curr_day_df], ignore_index=True)
    
combined_geotweet_df = combined_geotweet_df.rename(columns={"id_x": "tweet_id"})
del combined_geotweet_df['Unnamed: 0']
del combined_geotweet_df['Unnamed: 0.1']
del combined_geotweet_df['id_y']
combined_geotweet_df

,possibly_sensitive,entities,text,public_metrics,tweet_id,context_annotations,lang,in_reply_to_user_id,author_id,source,referenced_tweets,created_at,geo,hour_tweeted,referenced_tweet_ids,users_mentioned,place_id,latitude,longitude,day
0,False,NaN,That Indian corona music video is killing me😂,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1240727823830130688,"[{'domain': {'id': '123', 'name': 'Ongoing New...",en,NaN,2306150944,Twitter for iPhone,NaN,2020-03-19T19:52:19.000Z,{'place_id': '00e55e2b4c491c5f'},19,[],[],00e55e2b4c491c5f,9.073664,7.424094,1
1,False,"{'hashtags': [{'start': 123, 'end': 136, 'tag'...",Some of una never stop to de kiss for Cinema? ...,"{'retweet_count': 0, 'reply_count': 2, 'like_c...",1240731240719884288,"[{'domain': {'id': '123', 'name': 'Ongoing New...",en,NaN,801892591285506053,Twitter for Android,NaN,2020-03-19T20:05:54.000Z,{'place_id': '00e55e2b4c491c5f'},20,[],[],00e55e2b4c491c5f,9.073664,7.424094,1
2,False,"{'annotations': [{'start': 28, 'end': 39, 'pro...",Noooo... Its not because of Uncle Corona ooooo...,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1240732480883302400,"[{'domain': {'id': '123', 'name': 'Ongoing New...",en,NaN,159033468,Twitter for Android,"[{'type': 'quoted', 'id': '1240711779384094721'}]",2020-03-19T20:10:49.000Z,{'place_id': '00e55e2b4c491c5f'},20,['1240711779384094721'],[],00e55e2b4c491c5f,9.073664,7.424094,1
3,False,"{'annotations': [{'start': 29, 'end': 35, 'pro...",Religion Still Proving to be Nigeria's greates...,"{'retweet_count': 1, 'reply_count': 2, 'like_c...",1240732545186181121,"[{'domain': {'id': '123', 'name': 'Ongoing New...",en,NaN,702160979,Twitter for Android,NaN,2020-03-19T20:11:05.000Z,{'place_id': '00e55e2b4c491c5f'},20,[],[],00e55e2b4c491c5f,9.073664,7.424094,1
4,False,"{'annotations': [{'start': 4, 'end': 32, 'prob...",The Federal Ministry of Education has ordered ...,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1240733560513925120,"[{'domain': {'id': '123', 'name': 'Ongoing New...",en,NaN,139432369,Twitter for Android,NaN,2020-03-19T20:15:07.000Z,{'place_id': '00e55e2b4c491c5f'},20,[],[],00e55e2b4c491c5f,9.073664,7.424094,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1932055,False,"{'annotations': [{'start': 0, 'end': 11, 'prob...","Jason Kenney will be like ""I know a spot"" and ...","{'retweet_count': 1, 'reply_count': 0, 'like_c...",1281081426918518784,"[{'domain': {'id': '10', 'name': 'Person', 'de...",en,NaN,798748029578788864,Twitter for Android,NaN,2020-07-09T04:23:08.000Z,{'place_id': '0166f9add0e11684'},4,[],[],0166f9add0e11684,53.213836,-114.976718,112
1932056,False,"{'mentions': [{'start': 0, 'end': 5, 'username...",@Swil @jasonimms yeah normally i’d say 100% wa...,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1281081468026974208,"[{'domain': {'id': '123', 'name': 'Ongoing New...",en,765874.0,15902371,Tweetbot for Mac,"[{'type': 'replied_to', 'id': '128108072339953...",2020-07-09T04:23:17.000Z,{'place_id': '175d782c3dcf5528'},4,['1281080723399532544'],"['Swil', 'jasonimms']",175d782c3dcf5528,-42.882340,147.328557,112
1932057,False,"{'annotations': [{'start': 82, 'end': 103, 'pr...","Each species is a masterpiece, a creation asse...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",1281081587417956352,"[{'domain': {'id': '65', 'name': 'Interests an...",en,NaN,599021617,Instagram,NaN,2020-07-09T04:23:46.000Z,"{'place_id': '003f4a527524b7ee', 'coordinates'...",4,[],[],003f4a527524b7ee,-2.824641,36.032232,112
1932058,False,"{'hashtags': [{'start': 0, 'end': 5, 'tag': 'y...",#your winning Donny #drink the kool aid https:...,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1281081692183244800,"[{'domain': {'id': '123', 'name': 'Ongoing New...",en,NaN,27509605,Twitter for iPhone,"[{'type': 'quoted', 'id': '1281015120689324034'}]",2020-07-09T04:24:11.000Z,{'place_id': '56e6267bc475e580'},4,['1281015120689324034'],[],56e6267bc475e580,43.002204,-81.386664,112


#### Filter data; restrict tweet data to those posted from US

In [65]:
bottom_left_USA = [20, -135] # lat,long
top_right_USA = [50, -60]

total_filter = combined_geotweet_df[(combined_geotweet_df['latitude']>=bottom_left_USA[0]) & (combined_geotweet_df['latitude']<=top_right_USA[0]) & (combined_geotweet_df['longitude']>=bottom_left_USA[1]) & (combined_geotweet_df['longitude']<=top_right_USA[1])]
total_filter.to_csv('combined_geotweet_df_USA.csv')

#### Add region identifier column to combined dataframe

In [66]:
m,n = 30,30
lat_step = (top_right_USA[0] - bottom_left_USA[0])/m
lon_step = (top_right_USA[1] - bottom_left_USA[1])/n
regions = []

for _, row in total_filter.iterrows():
    lat_count = int((row['latitude']-bottom_left_USA[0])//lat_step)
    lon_count = int((row['longitude']-bottom_left_USA[1])//lon_step)
    identifier = '(' + str(lat_count) + ',' + str(lon_count) + ')'
    regions.append(identifier)
total_filter['region_identifiers'] = regions

<ipython-input-66-9f2301eefb4e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_filter['region_identifiers'] = regions


#### Add week column to combined dataframe

In [67]:
# week indexing starts at 0; day indexing start at 1
def get_week(day):
    return (day-1)//7

total_filter['week'] = total_filter['day'].apply(get_week)
total_filter

<ipython-input-67-221c75616274>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_filter['week'] = total_filter['day'].apply(get_week)


,possibly_sensitive,entities,text,public_metrics,tweet_id,context_annotations,lang,in_reply_to_user_id,author_id,source,...,geo,hour_tweeted,referenced_tweet_ids,users_mentioned,place_id,latitude,longitude,day,region_identifiers,week
38,False,NaN,fuck corona i miss my boyfriend,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",1240727871095631872,"[{'domain': {'id': '123', 'name': 'Ongoing New...",en,NaN,859271883279159297,Twitter for iPhone,...,{'place_id': 'fbd6d2f5a4e4a15e'},19,[],[],fbd6d2f5a4e4a15e,37.269176,-119.306607,1,"(17,6)",0
39,False,"{'mentions': [{'start': 0, 'end': 16, 'usernam...",@realDonaldTrump is a POS! https://t.co/3AswTG...,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1240741265064341505,"[{'domain': {'id': '10', 'name': 'Person', 'de...",en,2.507388e+07,781285034703491073,Twitter for iPhone,...,{'place_id': 'fbd6d2f5a4e4a15e'},20,['1240701140141879298'],['realDonaldTrump'],fbd6d2f5a4e4a15e,37.269176,-119.306607,1,"(17,6)",0
40,False,"{'annotations': [{'start': 57, 'end': 67, 'pro...",You know what we all need during this Corona o...,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1240745322952790016,"[{'domain': {'id': '10', 'name': 'Person', 'de...",en,NaN,2841636924,Twitter for iPhone,...,{'place_id': 'fbd6d2f5a4e4a15e'},21,[],[],fbd6d2f5a4e4a15e,37.269176,-119.306607,1,"(17,6)",0
41,False,NaN,This corona virus ain’t got shit over the vall...,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",1240753298140872705,"[{'domain': {'id': '123', 'name': 'Ongoing New...",en,NaN,888083561743384576,Twitter for iPhone,...,{'place_id': 'fbd6d2f5a4e4a15e'},21,[],[],fbd6d2f5a4e4a15e,37.269176,-119.306607,1,"(17,6)",0
42,False,"{'urls': [{'start': 12, 'end': 35, 'url': 'htt...",Corona who? https://t.co/VRJuZvGoaV,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1240759511423397889,"[{'domain': {'id': '123', 'name': 'Ongoing New...",en,NaN,2996168750,Twitter for iPhone,...,"{'place_id': 'fbd6d2f5a4e4a15e', 'coordinates'...",21,[],[],fbd6d2f5a4e4a15e,37.269176,-119.306607,1,"(17,6)",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1932049,False,"{'mentions': [{'start': 14, 'end': 25, 'userna...",@TheMagicianx @sandra8301 Thanks For That Expl...,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",1281080809617850368,"[{'domain': {'id': '123', 'name': 'Ongoing New...",en,2.914653e+09,822927813347069953,Twitter for Android,...,{'place_id': '0fc2aa81c294c001'},4,['1281008950738067458'],['sandra8301'],0fc2aa81c294c001,34.313591,-77.737552,112,"(14,22)",15
1932050,False,"{'annotations': [{'start': 31, 'end': 33, 'pro...",Here's News We Expected. Thank God This Rally ...,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",1281080894086774784,NaN,en,NaN,3309219404,Twitter for Android,...,{'place_id': 'd4157f8e4bd01598'},4,[],[],d4157f8e4bd01598,32.503280,-94.731341,112,"(12,16)",15
1932052,False,NaN,So Covid cases are higher than ever yet everyt...,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",1281081002769764352,"[{'domain': {'id': '123', 'name': 'Ongoing New...",en,NaN,16078427,Twitter for iPhone,...,{'place_id': '46eefbd4521bc929'},4,[],[],46eefbd4521bc929,39.577368,-76.998297,112,"(19,23)",15
1932053,False,"{'urls': [{'start': 118, 'end': 141, 'url': 'h...","New coronavirus projection predicts 208,000 US...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",1281081195904843778,"[{'domain': {'id': '123', 'name': 'Ongoing New...",en,NaN,20676553,Twitter for Android,...,{'place_id': '6d10b956d2e0f446'},4,[],[],6d10b956d2e0f446,35.373698,-86.222084,112,"(15,19)",15


#### Create dataframe to store counts by region and week

In [68]:
region_count_by_week = []
for week in range(total_weeks):
    temp_df = pd.DataFrame(total_filter[total_filter['week']==week]['region_identifiers'].value_counts())
    temp_df.reset_index(inplace=True)
    temp_df['week'] = week
    region_count_by_week.append(temp_df)
weekly_region_counts = pd.concat(region_count_by_week)
weekly_region_counts = weekly_region_counts.rename(columns={"index": "region", "region_identifiers": "count"})
weekly_region_counts.to_csv('weekly_region_counts.csv')